<img src="images/title.png">

<img src="images/IFR_geometry.png" width=600>

In [3]:
using SpecialFunctions

In [4]:
besseljprime(ν, x) = (besselj(ν - 1, x) - besselj(ν + 1, x))/2

besseljprime (generic function with 1 method)

In [5]:
hankelh2prime(ν, x) = (hankelh2(ν - 1, x) - hankelh2(ν + 1, x))/2

hankelh2prime (generic function with 1 method)

<img src="images/IFRE.png" width=600>

In [6]:
function IFRE(w, α)
    # w is the support element width in wavelengths
    arg = π*w*cos(α)
    sum = besselj(0, arg)/hankelh2(0, arg)
    prev = 0.0 + 0.0im
    for i in 1:100 
        sum += besselj(i, arg)/hankelh2(i, arg) + besselj(-i, arg)/hankelh2(-i, arg)
        change = abs(sum)/abs(prev)
        change = change < 1.0 ? 1/change : change
#        println(@sprintf("%d %.6f", i, change))
        if change < (1.0 + 1e-6)
            break
        end
        prev = sum
    end
    return -sum/arg
end
    
        
        

IFRH (generic function with 1 method)

In [7]:
IFRH(15.0,0.0)

-1.0381064894409588 + 0.06623515641096468im

<img src="images/IFRH.png" width=600>

In [8]:
function IFRH(w, α)
    # w is the support element width in wavelengths
    arg = π*w*cos(α)
    sum = besseljprime(0, arg)/hankelh2prime(0, arg)
    prev = 0.0 + 0.0im
    for i in 1:100 
        sum += besseljprime(i, arg)/hankelh2prime(i, arg) + besseljprime(-i, arg)/hankelh2prime(-i, arg)
        change = abs(sum)/abs(prev)
        change = change < 1.0 ? 1/change : change
#        println(@sprintf("%d %.6f", i, change))
        if change < (1.0 + 1e-6)
            break
        end
        prev = sum
    end
    return -sum/arg
end


IFRE (generic function with 1 method)

In [9]:
IFRE(0.2,0.0)

-0.291523568922654 - 0.44813685565826555im

<img src="images/IFRplot.png" width=400>

In [10]:
gaussian_taper(r, width, taper) = exp(-r^2*log(taper)/-width^2)

gaussian_taper (generic function with 1 method)

In [11]:
gaussian_taper(5,10,0.001)

0.1778279410038923

In [12]:
#using Plots;pyplot()

In [13]:
#using Cubature

In [14]:
#hquadrature(cos, 0.5, 1)[1]

In [15]:
#plot(linspace(0,10,500), 10*log10.(gaussian_taper.(linspace(0,10,500), 10, 0.001)))

integral_0^1 exp(-(((x1 - x0) t + x0)^2 + ((y1 - y0) t + y0)^2)/k) dt = (sqrt(π) sqrt(k) (erf((-x0 x1 + x1^2 - y0 y1 + y1^2)/(sqrt(k) sqrt((x0 - x1)^2 + (y0 - y1)^2))) - erf((-x0^2 + x0 x1 + y0 (y1 - y0))/(sqrt(k) sqrt((x0 - x1)^2 + (y0 - y1)^2)))) exp(-(x1 y0 - x0 y1)^2/(k ((x0 - x1)^2 + (y0 - y1)^2))))/(2 sqrt((x0 - x1)^2 + (y0 - y1)^2))

(Sqrt[k] Sqrt[Pi] (Erf[(-(x0 x1) + x1^2 - y0 y1 + y1^2)/(Sqrt[k] Sqrt[(x0 - x1)^2 + (y0 - y1)^2])] - Erf[(-x0^2 + x0 x1 + y0 (-y0 + y1))/(Sqrt[k] Sqrt[(x0 - x1)^2 + (y0 - y1)^2])]))/(2 E^((x1 y0 - x0 y1)^2/(k ((x0 - x1)^2 + (y0 - y1)^2))) Sqrt[(x0 - x1)^2 + (y0 - y1)^2])

$$\int_0^1 \exp{\biggl(-\frac{((x_1 - x_0) t + x_0)^2 + ((y_1 - y_0) t + y_0)^2}{k}}\biggr) \mathrm{dt} =$$

$$\biggl[ \sqrt{\pi k} \biggl(\mathrm{erf}\biggl(\frac{x_1^2 - x_0 x_1 - y_1(y_0 - y_1)}{ \sqrt{k((x_0 - x_1)^2 + (y_0 - y_1)^2)}}\biggr) - \mathrm{erf}\biggl(\frac{-x_0^2 + x_0 x_1 + y_0 (y_1 - y_0)}{\sqrt{k((x_0 - x_1)^2 + (y_0 - y_1)^2)}}\biggr)\biggr) \exp\biggl(\frac{-(x_1 y_0 - x_0 y_1)^2}{k ((x_0 - x_1)^2 + (y_0 - y_1)^2)}\biggr)\biggr] \bigg/2 \sqrt{(x_0 - x_1)^2 + (y_0 - y_1)^2}$$


In [ ]:
function integrated_taper(x0, x1, y0, y1, t, k)
    int = (sqrt(π*k)*(erf((x1^2 - x0*x1 - y1*(y0-y1))/sqrt(k*((x0 - x1)^2 + (y0 - y1)^2)) - erf(xxx)))

In [16]:
function read_wrl_data(filename)
    f = open(filename)
    coord = Float64[]
    index = Int64[]
    state = :None
    for ln in eachline(f)
        if contains(ln, "Coordinate")
            state = :Coord
        elseif contains(ln, "Index")
            state = :Index
        end
    
        if state == :Coord
            m = matchall(r"([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)", ln)
            if length(m) == 3
                push!(coord, parse.(Float64, m)...)
            end
        elseif state == :Index
            m = match(r"(\d+) (\d+) -1,",ln)
            if typeof(m) == RegexMatch && length(m.captures) == 2
                push!(index, parse.(Int64, m.captures)...)
            end
        end
    end
    close(f)
    coord = reshape(coord, 3, length(coord)÷3)
    index = reshape(index, 2, length(index)÷2)
    return(coord, index)
end

read_wrl_data (generic function with 1 method)

In [17]:
coord,ind=read_wrl_data("foo.wrl")

([0.894427 0.831052 … 0.138197 0.361803; -1.11022e-16 0.238856 … -0.425325 -0.587785; 0.447214 0.502295 … -0.894427 -0.723607], [91 0 … 143 60; 119 47 … 149 61])

In [28]:
segments = Array{Float64,2}[]
for i in 1:size(ind,2)
    push!(segments, hcat(coord[:, ind[1,i]+1],coord[:, ind[2,i]+1]))
end


In [29]:
ind

2×480 Array{Int64,2}:
  91   0   81  71  76  117  71   9  …  141  141   84  38  38   74  143  60
 119  47  113  75  77  150  72  49     144  142  118  45  41  146  149  61

In [30]:
segments[6]

3×2 Array{Float64,2}:
 -0.688191  -0.638197
  0.5        0.262866
 -0.525731  -0.723607

In [31]:
segments[6][3,1]

-0.525731112119

In [32]:
using PyPlot; pygui(true)

true

In [33]:
for s in segments
    if (s[3,1] < -0.1) || (s[3,2] < -0.1)
        continue
    end
#    text3D(c[1, i[1,n]+1], c[2, i[1,n]+1], c[3, i[1,n]+1], @sprintf("%d", i[1,n]))
    plot3D(s[1,:], s[2,:], s[3,:], color="green")
end


In [34]:
using Rotations

In [35]:
RotXYZ(-0.5,0.3,1.1)*[1,3,3]

3-element SVector{3,Float64}:
 -1.23431
  3.66488
  2.01124

In [36]:
using Distances

In [37]:
segments[1]

3×2 Array{Float64,2}:
 -0.687157  -0.809017   
 -0.681718  -0.587785   
  0.251148  -3.43078e-17

In [54]:
@time evaluate(Euclidean(),segments[10][:,2], zeros(3))

  0.000122 seconds (9 allocations: 432 bytes)


1.0000000000002973

In [50]:
using BenchmarkTools

In [55]:
@time sqrt(sum(abs2,segments[10][:,2]))

  0.000023 seconds (8 allocations: 320 bytes)


1.0000000000002973

In [ ]:
rotated = Array{Float64,2}[]
for s in segments
    rotated_mat = similar(s)
    rotated_mat[:,1] = RotXYZ(a,b,c)*s[:,1]
    rotated_mat[:,2] = RotXYZ(a,b,c)*s[:,2]
    if rotated_mat[3,1] > 0 && rotated_mat[3,2] > 0
        #sum(abs2, (I.I .- ctr)
        #subr
        #rim
        push!(rotated, rotated_mat)
    end
end

In [ ]:
# within circle

<img src="images/loss.png">

<img src="images/subreflector.png">

<img src="images/IFReffect.png">

<img src="images/illumination_weight.png">

<img src="images/CP.png">